In [ ]:
import datetime as dt
from datetime import datetime, timedelta, timezone, tzinfo
import joblib
import pytz
import json
import wallaroo
import wallaroo.assay_config
from wallaroo.object import EntityNotFoundError

import wallaroo.assay
from wallaroo.assay_config import BinMode, Aggregation, Metric

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import seaborn as sns
import requests
import uuid

plt.rcParams["figure.figsize"] = (12,6)
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
def status_color(status):
    if status == "Ok":
        return "green"
    elif status == "Warning":
        return "orange"
    else:
        return "red"
    
def create_legend():
    ok_patch = mpatches.Patch(color='green', label='Status Ok')
    warning_patch = mpatches.Patch(color='orange', label='Warning')
    alert_patch = mpatches.Patch(color='red', label='Alert')
    plt.legend(handles=[ok_patch, warning_patch, alert_patch])
    
def pick_colors(s):
    return [status_color(status) for status in s]

def extract_arrow_prediction(t):
    return t['out']['dense_2'][0]

def extract_arrow_prediction_nested(t):
    return t['out.dense_2'][0][0]

In [ ]:
import os
os.environ['DEPLOYMENT_CONFIG'] = json.dumps({"cpus": 0.1, "replica_count": 1, "memory": "100Mi"})
if os.environ.get("WALLAROO_SDK_CREDENTIALS"):
    auth_type="user_password"
else:
    auth_type="none"
auth_type = "sso"
client = wallaroo.Client(auth_type=auth_type, request_timeout=90, interactive=True)

In [ ]:
# Deploy a model/pipeline

rand_id = f"{np.random.randint(10000):05d}"
pipeline_name = 'modelinsightse2e' + rand_id
model_name = 'baseline' + rand_id

onnx_file_model_name = "house_price_keras.onnx"

fraud = (client.upload_model(model_name, 
                             onnx_file_model_name, 
                             framework=wallaroo.framework.Framework.ONNX)
                             .configure(tensor_fields=["tensor"])
        )
    
pipeline = client.build_pipeline(pipeline_name)
pipeline = pipeline.add_model_step(fraud)
deployment = pipeline.deploy()

In [ ]:
topic = client.get_topic_name(pipeline.id())
topic

In [ ]:
#Load some test data for inference
X_val = joblib.load("X_val.pkl")
assert isinstance(X_val, np.ndarray)

### Do some test inferences

In [ ]:
num_test_inferences = 1000
inference_start = dt.datetime.now()
for i in range(num_test_inferences):
    data = pd.DataFrame.from_dict({"tensor": [X_val[i].tolist()]})
    res = deployment.infer(data)

In [ ]:
inference_start

### check that we can query that data 

## Note: set correct dates

In [ ]:

day1 = dt.datetime(2023, 10,10, 0, 0, 0, 0, pytz.UTC)
day2 = dt.datetime(2023, 10,12, 0, 0, 0, 0, pytz.UTC)

In [ ]:
pipeline.logs(start_datetime=day1, end_datetime=day2, dataset=["time", "in", "out", "metadata", "check_failures"])

### Read baseline data from csv and extract the column that you want the assay to analyze.

In [ ]:
baseline_data = pd.read_csv("baseline_data.csv")
baseline_np_array = baseline_data["output_dense_2_0"].to_numpy()
baseline_np_array

In [ ]:
# iopath is required
# either provide baseline_data or baseline start and end dates.
assay_name = f"Test Assay test2"
assay_builder = client.build_assay(assay_name, pipeline, model_name, iopath="output dense_2 0", baseline_start=day1, baseline_end=day2)

In [ ]:
assay_builder.baseline_dataframe()

In [ ]:
assay_builder.baseline_histogram()

In [ ]:
assay_builder.baseline_kde()

In [ ]:
assay_config = assay_builder.build()
print(assay_config.to_json())

In [ ]:
assay_id = assay_builder.upload()
assay_id

In [ ]:
ardf = assay_config.interactive_run()
ardf

In [ ]:
ardf[0].__dict__

### Building assays with baseline_start and baseline_end dates

In [ ]:
assay_name = f"Test Assay 2222"
assay_builder = client.build_assay(assay_name, pipeline, model_name, "output dense_2 0", day1, day2)
assay_builder.__dict__

In [ ]:
assay_config = assay_builder.build()
print(assay_config.to_json())

In [ ]:
assay_id = assay_builder.upload()
assay_id

In [ ]:
ardf = assay_config.interactive_run()
ardf

In [ ]:
ardf[0].__dict__

In [ ]:
pipeline.undeploy()